In [2]:
import numpy as np
import math
from scipy.stats import unitary_group
from skimage.measure import block_reduce
from opt_einsum import contract
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
import torch
import time
import sys
import matplotlib.pyplot as plt
from matplotlib import rc
from src.QDDPM_torch import DiffusionModel, QDDPM, naturalDistance
#import src.ImageEncode as ie
rc('text', usetex=False)
rc('axes', linewidth=3)

Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [6]:
#note: maximum allowed value in image is 1
def angle_encode(image: np.array):
    Npixels = np.size(image)

    image = image.flatten()

    #if ((Npixels & (Npixels - 1)) == 0 and Npixels != 0):
    #    print("Pixel number is not a power of 2")
    #    return -1
    states = (1 + 1j) * np.zeros(2 ** Npixels)
    #for testing encode from 0 - pi/2 
    scale_val = (0.5 * math.pi) #/ np.max(image))
    image = image * scale_val

    for i in range(0, 2 ** Npixels):
        states[i] = 1 + 0j
        temp = i
        for j in range(0, Npixels):
            if (temp % 2):
                states[i] *= math.sin(image[Npixels - 1 - j])
            else:
                states[i] *= math.cos(image[Npixels - 1 - j])
            temp = int(temp / 2)
    
    return scale_val, states

In [ ]:
#for only 0-pi/2
def angle_decode(states: np.array, scale_val: float):
    Npixels = int(math.log2(np.size(states)))
    image_dim = int(Npixels ** 0.5)
    num_states = np.size(states)

    #if(image_dim != (Npixels ** 0.5)):
    #    print("image is not n by n pixels")
    #    return -1

    image_flattened = np.array([])
    image_nxn = np.zeros((image_dim, image_dim))

    #get rid of complex parts and square the array to get sin^2 and cos^2 components
    states_squared = np.square(np.abs(states))

    #create intermediate array to do calculations
    temp_copy = np.array(states_squared)

    while(np.size(image_flattened) < Npixels):
        if(np.size(temp_copy) == 2):
            #first elements of temp_copy will always be cos^2(x * scale_val), append x to image_flattened
            #add_angle = 0
            #if(temp_copy[0] >= 0 and temp_copy[1] < 0): #if in 4th quadrant
            #    add_angle = 3 * math.pi / 2
            #elif(temp_copy[0] < 0 and temp_copy[1] < 0): #if in third quadrant
            #    add_angle = math.pi / 2
            
            image_flattened = np.append(image_flattened, (math.acos((temp_copy[0]) ** 0.5))/ scale_val)

            #will have to divide the known values from the remaining states
            divisor_cos = temp_copy[0]

            #only copy half the remaining states, avoid dividing by zero
            if(image_flattened[-1] != 1):
                temp_copy = states_squared[0:int(np.size(states_squared)/2)] / divisor_cos
            else:
                temp_copy = states_squared[int(np.size(states_squared)/2): np.size(states_squared)]
            states_squared = np.array(temp_copy)

        while(np.size(temp_copy) > 2):
            #add neighboring array values to cancel out LSQB (sin^2 + cos^2 = 1)
            for j in range(0, int(np.size(temp_copy)/2)):
                temp_copy[j] += temp_copy[j+1]
                temp_copy = np.delete(temp_copy, j+1)

    for i in range(0, image_dim):
        for j in range(0, image_dim):
            image_nxn[i][j] = image_flattened[i * image_dim + j]
    

    return image_nxn, image_flattened

In [ ]:
def hamiltonian_decode(states: np.array, scale_val: float):
    Npixels = int(math.log2(np.size(states)))
    image_dim = int(Npixels ** 0.5)
    num_states = np.size(states)

    image_flattened = np.zeros(num_states)
    image_nxn = np.zeros((image_dim, image_dim))

    #un_normalize = num_states ** 0.5
    #states *= un_normalize
    for i in range(0, num_states):
        image_flattened[i] = np.angle([states[i]])
        print(image_flattened[i])
        if(image_flattened[i] < 0):
            image_flattened[i] += 2 * math.pi
    
    for i in range(0, image_dim):
        for j in range(0, image_dim):
            image_nxn[i][j] = image_flattened[i * image_dim + j]
    
    return image_nxn, image_flattened


In [15]:
#test angle_encode and decode
temp_test = np.array([1, 0.341, 0.282, 0.142, 0.109])
#plt.imshow(temp_test, cmap='grey',interpolation = 'nearest')
scale_val, states = angle_encode(temp_test)
#print(states)
decoded_nxn, decoded_flattened = angle_decode(states, scale_val)
print(decoded_flattened)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


C:\Users\George Zia\AppData\Local\Temp\ipykernel_21364\4062211737.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  image_flattened[i] = np.angle([states[i]])


In [38]:
#decode a set of states
decoded_nxn, decoded_flattened = angle_decode([0, 0, 0, 0, 1/4 , 2/4, math.sqrt(2)/4, 3/4], 0.5 * math.pi)
print(decoded_flattened)
#re-encode and check
scale_val, states = angle_encode(decoded_flattened)
print(states)

[1.         0.62235729 0.70483276]
[1.53080850e-17+0.j 3.06161700e-17+0.j 2.27055593e-17+0.j
 4.54111187e-17+0.j 2.50000000e-01+0.j 5.00000000e-01+0.j
 3.70809924e-01+0.j 7.41619849e-01+0.j]


In [3]:
print(np.angle([-1j]))

[-1.57079633]
